# ETL Process

Use this notebook to develop the ETL process for each of your tables before completing the **etl.py** file to load the whole datasets.

In [1]:
# import required modules
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
# get connect to postgresql database
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    # CREATE A CURSOR OBJECT
    cur = conn.cursor()
except psycopg2.Error as e:
    print(e)

In [3]:
# define function get all data files
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
    
    return all_files        

## process `song_data`

In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the all in this list
- Read the song file and view the data

In [4]:
# get all song files data
song_file_path = 'data/song_data'
song_file_path = get_files(song_file_path)
song_dfs = [] # create empty list to store the data frames
for file in song_file_path:
    df = pd.read_json(file, lines=True) # read data fram from json
    song_dfs.append(df) # append the data frame to the list

# concatenate all the data frames in the list.
song_df = pd.concat(song_dfs, ignore_index=True)
song_df


,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
1,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984
2,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
3,1,AR10USD1187B99F3F1,NaN,NaN,"Burlington, Ontario, Canada",Tweeterfriendly Music,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0
4,1,ARGSJW91187B9B1D6B,35.21962,-80.01955,North Carolina,JennyAnyKind,SOQHXMF12AB0182363,Young Boy Blues,218.77506,0
...,...,...,...,...,...,...,...,...,...,...
66,1,ARAGB2O1187FB3A161,NaN,NaN,,Pucho & His Latin Soul Brothers,SOLEYHO12AB0188A85,Got My Mojo Workin,338.23302,0
67,1,AR62SOJ1187FB47BB5,NaN,NaN,,Chase & Status,SOGVQGJ12AB017F169,Ten Tonne,337.68444,2005
68,1,ARBEBBY1187B9B43DB,NaN,NaN,"Gainesville, FL",Tom Petty,SOFFKZS12AB017F194,A Higher Place (Album Version),236.17261,1994
69,1,ARNNKDK1187B98BBD5,45.80726,15.96760,Zagreb Croatia,Jinx,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),407.37914,0


In [5]:
# check all attributes datatype
song_df.dtypes

num_songs             int64
artist_id            object
artist_latitude     float64
artist_longitude    float64
artist_location      object
artist_name          object
song_id              object
title                object
duration            float64
year                  int64
dtype: object

### #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the all record in the dataframe
- Convert the array to a list and set it to `song_data`

In [6]:
song_data = []
for i, row in song_df.iterrows():
    song_data.append(list(row[['song_id','title','artist_id','year','duration']].values))
song_data
#first_song_data = list(df_first[['song_id','title','artist_id','year','duration']].values[0])
#first_song_data

[['SONHOTT12A8C13493C',
  'Something Girls',
  'AR7G5I41187FB4CE6C',
  1982,
  233.40363],
 ['SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984, 269.81832],
 ['SOFSOCN12A8C143F5D',
  'Face the Ashes',
  'ARXR32B1187FB57099',
  2007,
  209.60608],
 ['SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0, 189.57016],
 ['SOQHXMF12AB0182363', 'Young Boy Blues', 'ARGSJW91187B9B1D6B', 0, 218.77506],
 ['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546],
 ['SOMZWCG12A8C13C480',
  "I Didn't Mean To",
  'ARD7TVE1187B99BFB1',
  0,
  218.93179],
 ['SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA', 0, 177.47546],
 ['SOUDSGM12AC9618304',
  'Insatiable (Instrumental Version)',
  'ARNTLGG11E2835DDB9',
  0,
  266.39628],
 ['SOYMRWW12A6D4FAB14',
  'The Moon And I (Ordinary Day Album Version)',
  'ARKFYS91187B98E58F',
  0,
  267.7024],
 ['SOMJBYD12A6D4F8557',
  'Keepin It Real (Skit)',
  'ARD0S291187B9B7BF5',
  0,
  114.78159],
 ['SOVYKGO12AB0187199',

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [7]:
for data in song_data:
    cur.execute(song_table_insert, data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

### #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the all record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [8]:
artist_data = []
for i, row in song_df.iterrows():
    artist_data.append(list(row[['artist_id', 'artist_name', 'artist_location', 
                              'artist_latitude', 'artist_longitude']].values))
artist_data


[['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan],
 ['AR8ZCNI1187B9A069B', 'Planet P Project', '', nan, nan],
 ['ARXR32B1187FB57099', 'Gob', '', nan, nan],
 ['AR10USD1187B99F3F1',
  'Tweeterfriendly Music',
  'Burlington, Ontario, Canada',
  nan,
  nan],
 ['ARGSJW91187B9B1D6B', 'JennyAnyKind', 'North Carolina', 35.21962, -80.01955],
 ['ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892],
 ['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan],
 ['ARKRRTF1187B9984DA', 'Sonora Santanera', '', nan, nan],
 ['ARNTLGG11E2835DDB9', 'Clp', '', nan, nan],
 ['ARKFYS91187B98E58F', 'Jeff And Sheri Easter', '', nan, nan],
 ['ARD0S291187B9B7BF5', 'Rated R', 'Ohio', nan, nan],
 ['ARH4Z031187B9A71F2', 'Faye Adams', 'Newark, NJ', 40.73197, -74.17418],
 ['ARB29H41187B98F0EF', 'Terry Callier', 'Chicago', 41.88415, -87.63241],
 ['AR0RCMP1187FB3F427',
  'Billie Jo Spears',
  'Beaumont, TX',
  30.08615,
  -94.10158],
 ['AR47JEX1187B995D81', 'SUE THOMPSON', 'Nevada,

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [9]:
for data in artist_data:
    cur.execute(artist_table_insert,data)
    conn.commit()

## Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the all log file in this list
- Read the log file and view the data

In [10]:
# get all song files data
log_file_path = 'data/log_data'
log_file_path = get_files(log_file_path)
log_dfs = [] # create empty list to store the data frames
for file in log_file_path:
    df = pd.read_json(file, lines=True) # read data fram from json
    log_dfs.append(df) # append the data frame to the list

# concatenate all the data frames in the list.
log_df = pd.concat(log_dfs, ignore_index=True)
log_df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8051,Motion City Soundtrack,Logged In,Stefany,F,1,White,205.26975,free,"Lubbock, TX",PUT,NextSong,1.540708e+12,867,Fell In Love Without You (Acoustic),200,1543092353796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
8052,Ticanaf,Logged In,Stefany,F,2,White,2594.87302,free,"Lubbock, TX",PUT,NextSong,1.540708e+12,867,The Thousand Names of Lord Shiva (Part 1),200,1543092558796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
8053,The Far East Movement,Logged In,Ryan,M,0,Smith,228.20526,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,899,Dance Like Michael Jackson,200,1543097750796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
8054,the bird and the bee,Logged In,Ryan,M,1,Smith,189.51791,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,899,You're A Cad,200,1543097978796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


### #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [11]:
# filter the log datafram with 'page'= 'NextSong' and check the log data null values
log_df = log_df[log_df['page']=='NextSong']
print(log_df.isna().sum())
print('\n')
print('The total number of log data is: ',len(log_df))

artist           0
auth             0
firstName        0
gender           0
itemInSession    0
lastName         0
length           0
level            0
location         0
method           0
page             0
registration     0
sessionId        0
song             0
status           0
ts               0
userAgent        0
userId           0
dtype: int64


The total number of log data is:  6820


In [12]:
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [13]:
dt = pd.to_datetime(log_df['ts'], unit='ms')
dt.head()

0   2018-11-11 02:33:56.796
1   2018-11-11 02:36:10.796
2   2018-11-11 02:40:34.796
4   2018-11-11 04:36:13.796
5   2018-11-11 04:36:46.796
Name: ts, dtype: datetime64[ns]

In [14]:
# Extract the timestamp, hour, day, week of year, month, year, and weekday
timestamp = log_df['ts'].values
hour = dt.dt.hour.values
day = dt.dt.day.values
week_of_year = dt.dt.isocalendar().week.values
month = dt.dt.month.values
year = dt.dt.year.values
weekday = dt.dt.weekday.values


# generate time data list
time_data = [timestamp,hour,day,week_of_year,month,year,weekday]

# generate column list
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [15]:
# combining column_labels and time_data into a dictionary and converting this into a dataframe
time_dic = dict(zip(column_labels, time_data))
time_df = pd.DataFrame.from_dict(time_dic)
time_df

,start_time,hour,day,week,month,year,weekday
0,1541903636796,2,11,45,11,2018,6
1,1541903770796,2,11,45,11,2018,6
2,1541904034796,2,11,45,11,2018,6
3,1541910973796,4,11,45,11,2018,6
4,1541911006796,4,11,45,11,2018,6
...,...,...,...,...,...,...,...
6815,1543092353796,20,24,47,11,2018,5
6816,1543092558796,20,24,47,11,2018,5
6817,1543097750796,22,24,47,11,2018,5
6818,1543097978796,22,24,47,11,2018,5


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [16]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, row)
    conn.commit()

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [17]:
user_df = log_df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [18]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

### #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [19]:
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [24]:
for index, row in log_df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    result = cur.fetchone()

    if result:
        songid, artistid = result
    else:
        songid, artistid = None, None
    

    # insert songplay record
    songplay_data = [row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]
    
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
    

## Close Connection to Sparkify Database

In [26]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.